In [1]:
import os
import re
import time
import datetime
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as Soup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

In [2]:
# Configure ChromeDriver
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
chromedriver_path = r'chromedriver-win64\chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

In [3]:
def clean_text(text):
    """Utility function to remove newlines and extra spaces."""
    if not text:
        return ""
    return text.replace('\n', ' ').strip()

def accept_cookies():
    """Click the 'Accept all' button if the cookie consent dialog appears."""
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[contains(@class,"AIC7ge")]//span[@jsname="V67aGc"]'))
        )
        accept_button.click()
        print("Clicked the 'Accept all' button.")
    except TimeoutException:
        print("Cookie consent button not found.")

def extract_lat_long_from_url(url):
    """Extract latitude and longitude from the URL after the '@' sign."""
    latitude = ""
    longitude = ""
    if "@" in url:
        try:
            coords_str = url.split("@")[1].split("/")[0]  # e.g., "51.4293331,-2.5460859,17z"
            coords = coords_str.split(",")
            if len(coords) >= 2:
                latitude = coords[0].strip()
                longitude = coords[1].strip()
        except Exception as e:
            print("Error extracting lat long:", e)
    return latitude, longitude

In [4]:
def scrape_reviews_from_current_page_separated():
    try:
        reviews_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, 
                '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]')
            )
        )
        reviews_button.click()
        print("Clicked reviews button.")
    except TimeoutException:
        print("Reviews button not found.")
        return []
    
    pane_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]'
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, pane_xpath)))
    except TimeoutException:
        print("Reviews pane not found.")
        return []
    
    pane = driver.find_element(By.XPATH, pane_xpath)
    
    previous_review_count = 0
    no_change_iterations = 0
    max_no_change_iterations = 3  # break after 3 iterations with no new reviews
    
    while True:
        try:
            pane = driver.find_element(By.XPATH, pane_xpath)
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
        except StaleElementReferenceException:
            time.sleep(1)
            continue
        
        time.sleep(2)
        
        # After each swipe, click all visible "Read More" buttons.
        more_buttons = driver.find_elements(By.CLASS_NAME, 'w8nwRe')
        if more_buttons:
            print(f"Found {len(more_buttons)} 'Read More' button(s) in this swipe, clicking them...")
            for button in more_buttons:
                try:
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(0.5)
                except StaleElementReferenceException:
                    continue
        
        page_source = driver.page_source
        soup = Soup(page_source, "html.parser")
        user_divs = soup.select('div.GHT2ce.NsCY4')
        content_divs = soup.select('div.GHT2ce:not(.NsCY4)')
        current_review_count = min(len(user_divs), len(content_divs))
        print(f"Scrolled: Current review count = {current_review_count}")
        
        if current_review_count == previous_review_count:
            no_change_iterations += 1
            print(f"No new reviews detected, iteration {no_change_iterations} of {max_no_change_iterations}.")
            if no_change_iterations >= max_no_change_iterations:
                print("No new reviews loaded for several iterations. Breaking out of scroll loop.")
                break
        else:
            print(f"New reviews loaded: {current_review_count - previous_review_count}")
            previous_review_count = current_review_count
            no_change_iterations = 0

    print(f"Total reviews loaded after scrolling: {current_review_count}")
    
    reviews = []
    for user_div, content_div in zip(user_divs, content_divs):
        username_el = user_div.select_one('div.d4r55')
        username = clean_text(username_el.text) if username_el else "No username"
        
        star_el = content_div.select_one('span.kvMYJc')
        if star_el and star_el.has_attr('aria-label'):
            rating_str = clean_text(star_el['aria-label'])
            star_rating = rating_str.split()[0] if rating_str else "No rating"
        else:
            star_rating = "No rating"
        
        time_el = content_div.select_one('span.rsqaWe')
        review_time = clean_text(time_el.text) if time_el else "No date"
        
        text_el = content_div.select_one('span.wiI7pd')
        review_text = clean_text(text_el.text) if text_el else "No review text"
        
        reply_div = content_div.select_one('div.CDe7pd')
        if reply_div:
            reply_text_el = reply_div.select_one('div.wiI7pd')
            reply_text = clean_text(reply_text_el.text) if reply_text_el else "No reply"
            reply_time_el = reply_div.select_one('span.DZSIDd')
            reply_time = clean_text(reply_time_el.text) if reply_time_el else "No date"
        else:
            reply_text = "No reply"
            reply_time = "No date"
        
        like_btn = content_div.select_one('button.GBkF3d')
        if like_btn and like_btn.has_attr('aria-label'):
            likes_str = clean_text(like_btn['aria-label'])
            match = re.search(r'(\d+)', likes_str)
            likes = match.group(1) if match else "0"
        else:
            likes = "0"
        
        reviews.append((username, review_text, star_rating, review_time, reply_text, reply_time, likes))
    
    return reviews

In [5]:
def scrape_google_maps_reviews_multi(search_query, csv_address):
    driver.get('https://www.google.com/maps')
    # Cookies are accepted only once at the beginning.
    
    search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "searchboxinput")))
    search_box.clear()
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  # Wait for results to load.
    
    all_reviews_data = []
    
    result_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/place/")]')
    if len(result_links) == 0:
        print("Direct merchant page detected. Attempting to scrape reviews from current page.")
        page_url = driver.current_url
        latitude, longitude = extract_lat_long_from_url(page_url)
        # Extract merchant address from the direct page and remove "Address: " if present.
        address = 'No address found'
        try:
            address_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '//button[contains(@class,"CsEnBe") and @aria-label]'))
            )
            address = clean_text(address_element.get_attribute("aria-label")).replace("Address: ", "").strip()
        except TimeoutException:
            print("Address not found on direct merchant page.")
        reviews = scrape_reviews_from_current_page_separated()
        if reviews:
            for r in reviews:
                (username, review_text, star_rating, review_time, reply_text, reply_time, likes) = r
                all_reviews_data.append((search_query, csv_address, address, latitude, longitude,
                                         username, review_text, star_rating, review_time, reply_text, reply_time, likes))
        else:
            print(f"No reviews found for: {search_query}")
        return all_reviews_data

    merchant_index = 0
    while True:
        result_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/place/")]')
        current_count = len(result_links)
        
        if merchant_index < current_count:
            for i in range(merchant_index, current_count):
                link = result_links[i]
                href = link.get_attribute("href")
                if not href:
                    continue

                driver.execute_script("window.open(arguments[0]);", href)
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(3)  # Wait for page load.
                
                page_url = driver.current_url
                latitude, longitude = extract_lat_long_from_url(page_url)
                
                # Extract merchant address from aria-label of button with class "CsEnBe" and remove "Address: "
                address = 'No address found'
                try:
                    address_element = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '//button[contains(@class,"CsEnBe") and @aria-label]'))
                    )
                    address = clean_text(address_element.get_attribute("aria-label")).replace("Address: ", "").strip()
                except TimeoutException:
                    print("Address not found on this result.")
                
                try:
                    reviews = scrape_reviews_from_current_page_separated()
                    if reviews:
                        for r in reviews:
                            (username, review_text, star_rating, review_time, reply_text, reply_time, likes) = r
                            all_reviews_data.append((search_query, csv_address, address, latitude, longitude,
                                                     username, review_text, star_rating, review_time, reply_text, reply_time, likes))
                except Exception as e:
                    print("Error scraping reviews for one result:", e)
                
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(2)
            merchant_index = current_count
        else:
            try:
                divSideBar = driver.find_element(By.CSS_SELECTOR, "div[aria-label*='Results']")
                last_height = driver.execute_script("return arguments[0].scrollHeight", divSideBar)
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", divSideBar)
                time.sleep(2)
                new_height = driver.execute_script("return arguments[0].scrollHeight", divSideBar)
                if new_height == last_height:
                    print("No more new merchants loaded.")
                    break
                else:
                    print("Scrolled one page down.")
            except Exception as e:
                print("Error scrolling the sidebar:", e)
                break
            
            new_result_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/place/")]')
            new_count = len(new_result_links)
            if new_count > current_count:
                merchant_index = current_count
            else:
                break

    return all_reviews_data

In [6]:
addresses_csv = os.path.join('Results', 'puregym_addresses.csv')
addresses_df = pd.read_csv(addresses_csv)
total_entries = len(addresses_df)

# Open Google Maps and accept cookies only once at the beginning.
driver.get('https://www.google.com/maps')
accept_cookies()

output_dir = 'Results'
output_file_name = "Google_Map_Reviews_for_puregym_addresses.csv"
output_path = os.path.join(output_dir, output_file_name)
# Ensure the output directory exists.
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

start_time = time.time()

for idx, row in addresses_df.iterrows():
    url = str(row["url"]).strip()
    if url.endswith('/'):
        url = url[:-1]
    # Extract the segment from the URL.
    segment = url.rsplit('/', 1)[-1]
    # Replace hyphens with spaces and convert to title case.
    segment = segment.replace("-", " ").title()
    
    search_query = "PureGym " + segment
    # Get the CSV "address" column content.
    csv_address = clean_text(str(row["address"]))
    
    elapsed = time.time() - start_time
    avg_time = elapsed / (idx + 1)
    remaining_sec = avg_time * (total_entries - idx - 1)
    remaining_str = str(datetime.timedelta(seconds=int(remaining_sec)))
    
    print(f"Processing search for: {search_query} ({idx+1} of {total_entries}) - Estimated time remaining: {remaining_str}")
    
    reviews_data = scrape_google_maps_reviews_multi(search_query, csv_address)
    if reviews_data:
        # Save the results for the current business immediately.
        df = pd.DataFrame(
            reviews_data, 
            columns=[
                'Search Query', 'CSV Address', 'Merchant Address', 'Latitude', 'Longitude',
                'Username', 'Review', 'Rating', 'Review Time', 'Reply', 'Reply Time', 'Likes'
            ]
        )
        if not os.path.isfile(output_path):
            df.to_csv(output_path, mode='w', index=False, header=True, encoding='utf-8-sig')
        else:
            df.to_csv(output_path, mode='a', index=False, header=False, encoding='utf-8-sig')
        print(f"Saved results for {search_query}.")
    else:
        print(f"No reviews found for: {search_query}")

Clicked the 'Accept all' button.
Processing search for: PureGym Aberdare (1 of 442) - Estimated time remaining: 0:00:01
Direct merchant page detected. Attempting to scrape reviews from current page.
Reviews button not found.
No reviews found for: PureGym Aberdare
No reviews found for: PureGym Aberdare
Processing search for: PureGym Aberdeen Kittybrewster (2 of 442) - Estimated time remaining: 0:50:20
Direct merchant page detected. Attempting to scrape reviews from current page.
Clicked reviews button.
Found 6 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 10
New reviews loaded: 10
Found 6 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 20
New reviews loaded: 10
Found 10 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 30
New reviews loaded: 10
Found 9 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 40
New reviews loaded: 10
Found 6 

Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 90
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 100
New reviews loaded: 10
Found 5 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 110
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 120
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 130
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 140
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 150
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 160
New reviews loaded: 10
Found 10 'Read More' butt

Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 230
New reviews loaded: 10
Found 1 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 240
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 250
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 260
New reviews loaded: 10
Scrolled: Current review count = 270
New reviews loaded: 10
Scrolled: Current review count = 280
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 290
New reviews loaded: 10
Scrolled: Current review count = 300
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 310
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review c

Scrolled: Current review count = 10
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 20
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 30
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 40
New reviews loaded: 10
Found 8 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 50
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 60
New reviews loaded: 10
Found 6 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 70
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 80
New reviews loaded: 10
Found 1 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count =

Scrolled: Current review count = 130
New reviews loaded: 10
Scrolled: Current review count = 136
New reviews loaded: 6
Scrolled: Current review count = 136
No new reviews detected, iteration 1 of 3.
Scrolled: Current review count = 136
No new reviews detected, iteration 2 of 3.
Scrolled: Current review count = 136
No new reviews detected, iteration 3 of 3.
No new reviews loaded for several iterations. Breaking out of scroll loop.
Total reviews loaded after scrolling: 136
Saved results for PureGym Antrim.
Processing search for: PureGym Ashford Warren Retail Park (15 of 442) - Estimated time remaining: 9:02:28
Direct merchant page detected. Attempting to scrape reviews from current page.
Clicked reviews button.
Found 1 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 10
New reviews loaded: 10
Found 5 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 20
New reviews loaded: 10
Found 7 'Read More' button(s) in this s

...

Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 240
New reviews loaded: 10
Found 1 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 250
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 260
New reviews loaded: 10
Scrolled: Current review count = 270
New reviews loaded: 10
Found 1 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 280
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 290
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 300
New reviews loaded: 10
Found 1 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 310
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review

Scrolled: Current review count = 185
No new reviews detected, iteration 3 of 3.
No new reviews loaded for several iterations. Breaking out of scroll loop.
Total reviews loaded after scrolling: 185
Saved results for PureGym Wisbech.
Processing search for: PureGym Witney (433 of 442) - Estimated time remaining: 0:19:00
Direct merchant page detected. Attempting to scrape reviews from current page.
Clicked reviews button.
Found 8 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 10
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 20
New reviews loaded: 10
Found 7 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 30
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 40
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count =

Scrolled: Current review count = 100
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 110
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 120
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 130
New reviews loaded: 10
Scrolled: Current review count = 140
New reviews loaded: 10
Found 1 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 150
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 160
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 170
New reviews loaded: 10
Found 2 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 180
New reviews loaded: 10
Found 6 'Read More' button

Found 9 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 50
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 60
New reviews loaded: 10
Found 1 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 70
New reviews loaded: 10
Found 5 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 80
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 90
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 100
New reviews loaded: 10
Found 3 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 110
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 120
New reviews loaded: 10
Found 6 'Read More' button(s)

Found 11 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 20
New reviews loaded: 10
Found 11 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 30
New reviews loaded: 10
Found 10 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 40
New reviews loaded: 10
Found 10 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 50
New reviews loaded: 10
Found 10 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 60
New reviews loaded: 10
Found 10 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 70
New reviews loaded: 10
Found 4 'Read More' button(s) in this swipe, clicking them...
Scrolled: Current review count = 75
New reviews loaded: 5
Scrolled: Current review count = 75
No new reviews detected, iteration 1 of 3.
Scrolled: Current review count = 75
No new reviews detected, iteratio

In [7]:
driver.quit()